In [ ]:
library(Matrix)
library(Seurat)
library(SeuratObject)
library(dplyr)
library(ggplot2)
library(gridExtra)
library(DropletUtils)
library(patchwork)

## Load Data 

In [ ]:
int.data <- readRDS(file = "./data/integrated_data.RDS")

## Split into Individual Samples

In [ ]:
data_list <- list()
for (sample in unique(int.data$sample)){
    data_list[[sample]] <- subset(int.data, cells = rownames(int.data@meta.data[int.data@meta.data$sample == sample,]))
}

## Cluster Individual Samples

In [ ]:
for (sample in names(data_list)){
    DefaultAssay(data_list[[sample]]) <- "SCT"
    data_list[[sample]] <- FindVariableFeatures(data_list[[sample]])
    data_list[[sample]] <- RunPCA(data_list[[sample]], assay = "SCT", verbose = FALSE)
    data_list[[sample]] <- FindNeighbors(data_list[[sample]], reduction = "pca", dims = 1:30, verbose = FALSE)
    data_list[[sample]] <- RunUMAP(data_list[[sample]], reduction = "pca", dims = 1:30, verbose = FALSE)
    data_list[[sample]] <- FindClusters(data_list[[sample]], resolution = 0.3, verbose = FALSE)
}

In [ ]:
UMAPs_plots <- list()
Spatial_plots <- list()
Vln_Plots <- list()
for (sample in names(data_list)){
    UMAPs_plots[[sample]] <- DimPlot(data_list[[sample]])/DimPlot(data_list[[sample]], group.by = "integrated_snn_res.0.3")
    Spatial_plots[[sample]] <-  SpatialDimPlot(data_list[[sample]])/SpatialDimPlot(data_list[[sample]], group.by = "integrated_snn_res.0.3")  
    Vln_Plots[[sample]] <-  VlnPlot(data_list[[sample]], features = "Cancer_MS", group.by = "seurat_clusters")/VlnPlot(data_list[[sample]], features = "Cancer_MS", group.by = "integrated_snn_res.0.3")
                        
}

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 10)
wrap_plots(UMAPs_plots, ncol = length(UMAPs_plots))
wrap_plots(Spatial_plots,ncol = length(Spatial_plots)) 
wrap_plots(Vln_Plots,ncol = length(Vln_Plots))

## SubCluster cluster 8 Individually

In [ ]:
subs <- list()
for (sample in names(data_list)){
    subs[[sample]] <- subset(data_list[[sample]], subset = integrated_snn_res.0.3 == 8)
    subs[[sample]] <- RunPCA(subs[[sample]], assay = "SCT", verbose = FALSE)
    subs[[sample]] <- FindNeighbors(subs[[sample]], reduction = "pca", dims = 1:30, verbose = FALSE)
    subs[[sample]] <- RunUMAP(subs[[sample]], reduction = "pca", dims = 1:30, verbose = FALSE)
    subs[[sample]] <- FindClusters(subs[[sample]], resolution = 0.3, cluster.name = "cluster8_ind", verbose = FALSE)
}

In [ ]:
for (sample in names(subs)){
    subs[[sample]] <- AddModuleScore(object =  subs[[sample]], features = list("cin" = c('SERPINB13','SERPINB5','CLDN1','TP63','CDKN2A','TNS4','DSC3')), name = "CIN_genes", assay = "SCT", nbin = 5)             
    subs[[sample]] <- AddModuleScore(object =  subs[[sample]], features = list("normal" = c('KRT6C','GJB6','SBSN','KRTDAP','KRT6B')), name = "normal_genes", assay = "SCT", nbin = 5)             
}

In [ ]:
UMAPs_plots <- list()
Spatial_plots <- list()
CIN_VlnPlots <- list()
normal_VlnPlots <- list()
for (sample in names(data_list)){
    UMAPs_plots[[sample]] <- DimPlot(subs[[sample]], group.by = "cluster8_ind")/DimPlot(subs[[sample]], group.by = "SCT_snn_res.0.3")
    Spatial_plots[[sample]] <-  SpatialDimPlot(subs[[sample]], group.by = "cluster8_ind", crop = FALSE)/SpatialDimPlot(subs[[sample]], group.by = "SCT_snn_res.0.3", crop = FALSE)  
    CIN_VlnPlots[[sample]] <-  VlnPlot(subs[[sample]], features = "CIN_genes1", group.by = "cluster8_ind")/VlnPlot(subs[[sample]], features = "CIN_genes1", group.by = "SCT_snn_res.0.3")                   
    normal_VlnPlots[[sample]] <-  VlnPlot(subs[[sample]], features = "normal_genes1", group.by = "cluster8_ind")/VlnPlot(subs[[sample]], features = "normal_genes1", group.by = "SCT_snn_res.0.3")                   

}

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 10)
wrap_plots(UMAPs_plots, ncol = length(UMAPs_plots))
wrap_plots(Spatial_plots,ncol = length(Spatial_plots)) 
wrap_plots(CIN_VlnPlots,ncol = length(CIN_VlnPlots))
wrap_plots(normal_VlnPlots,ncol = length(normal_VlnPlots))

In [ ]:
subs[[1]]$match_annotation <- ifelse(subs[[1]]$cluster8_ind == 1, "CIN", "NON-SCC (p16+)")
subs[[2]]$match_annotation <- ifelse(subs[[2]]$cluster8_ind == 1, "SCC", "CIN")
subs[[3]]$match_annotation <- ifelse(subs[[3]]$cluster8_ind == 1, "CIN", "CIN")
subs[[4]]$match_annotation <- ifelse(subs[[4]]$cluster8_ind == 1, "NON-SCC", "CIN")
subs[[5]]$match_annotation <- ifelse(subs[[5]]$cluster8_ind == 2, "CIN", "NON-SCC")
subs[[6]]$match_annotation <- ifelse(subs[[6]]$cluster8_ind == 0, "NON-SCC", "CIN")
subs[[7]]$match_annotation <- ifelse(subs[[7]]$cluster8_ind == 0, "NON-SCC", "CIN")

## Add Annotations back to integrated object

In [ ]:
for (sample in names(subs)){
    data_list[[sample]]$cluster8_ind <- ifelse(
      rownames(data_list[[sample]]@meta.data) %in% rownames(subs[[sample]]@meta.data),
       paste0(subs[[sample]]$cluster8_ind[match(rownames(data_list[[sample]]@meta.data), rownames(subs[[sample]]@meta.data))],"_",sample),
      "Other")
}

In [ ]:
for (sample in names(subs)){
    data_list[[sample]]$cluster_annotations <- ifelse(
      rownames(data_list[[sample]]@meta.data) %in% rownames(subs[[sample]]@meta.data),
       subs[[sample]]$match_annotation[match(rownames(data_list[[sample]]@meta.data), rownames(subs[[sample]]@meta.data))],
      "Other")
}

In [ ]:
for (sample in names(subs)){
    data_list[[sample]]$cluster_de <- ifelse(
      rownames(data_list[[sample]]@meta.data) %in% rownames(subs[[sample]]@meta.data),
       subs[[sample]]$match_annotation[match(rownames(data_list[[sample]]@meta.data), rownames(subs[[sample]]@meta.data))],
      data_list[[sample]]$integrated_snn_res.0.3)
}

In [ ]:
## merge data objects
merge <- merge(data_list[[1]], data_list[2:length(data_list)])
int.data$individual_clustering <- merge$SCT_snn_res.0.3
int.data$cluster8_ind <- merge$cluster8_ind
int.data$cluster_annotations <- merge$cluster_annotations
int.data$cluster_de <- merge$cluster_de

In [ ]:
int.data$cluster_de <- ifelse(!int.data$cluster_de %in% c("CIN", "NON-SCC (p16+)","SCC", "NON-SCC"), (as.integer(int.data$cluster_de)-1), int.data$cluster_de)

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 8)
DimPlot(int.data , group.by = c("integrated_snn_res.0.3"))|DimPlot(int.data , group.by = c("cluster_de"), label = T)|FeaturePlot(int.data, features = "sct_CDKN2A", slot = "data")
FeaturePlot(int.data, features = "sct_SERPINB3", slot = "data")|FeaturePlot(int.data, features = "sct_TP63", slot = "data")|FeaturePlot(int.data, features = "sct_KRT5", slot = "data")

In [ ]:
options(repr.plot.width = 8, repr.plot.height =8)
p <- DimPlot(int.data , group.by = c("integrated_snn_res.0.3"))

ggsave(p, filename = "~/umap.pdf", height = 8, width = 8)

In [ ]:
sub <- subset(int.data, subset = integrated_snn_res.0.3 == 8)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 15)

p <- SpatialDimPlot(sub, group.by = "cluster8_ind", crop = F, pt.size.factor = 2, ncol = 3)
p
ggsave(p, filename = "~/CIN_subcluster_ind.pdf", height = 15, width = 20)

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 7)
p<-  DimPlot(sub, group.by = "cluster_de")

ggsave(p, filename = "~/CIN_subcluster_umap.pdf", height = 7, width = 7)

In [ ]:
source("https://raw.githubusercontent.com/agc888/Helper-Scripts-for-Bioinformatics/refs/heads/main/R%20Scripts/Pseudobulking_DE_Seurat.r")

In [ ]:
sub <- JoinLayers(sub, assay = "Spatial")
sub$check_subclusters <- ifelse(sub$cluster_de == "CIN", "C", ifelse(sub$cluster_de == "NON-SCC", "N", ifelse(sub$cluster_de == "SCC", "S","P"))) 
sub$check_subclusters <- paste0(sub$cluster8_ind, "_", sub$check_subclusters)

In [ ]:
edger_obj <- FindAllDEGs(sub, ident = "check_subclusters", n = 1)

In [ ]:
edger_obj$samples$type <- unlist(lapply(rownames(edger_obj$samples), function(x){strsplit(x, split = "_")[[1]][4]}))

edger_obj$samples$type <- ifelse(edger_obj$samples$type == "C", "CIN",
       ifelse(edger_obj$samples$type == "N", "NON-SCC",
             ifelse(edger_obj$samples$type == "S", "SCC", "NON-SCC(p16+)")))

In [ ]:
gene_markers <- list("cin" = c('SERPINB13','SERPINB5','CLDN1','TP63','CDKN2A','TNS4','DSC3'),
     "scc" = c('CASP14','PSORS1C2','DNAH17','SERPINB12','SLC44A5','TCHH','PNLDC1','DIAPH3','CPA4','CALML5' ),
     "normal" = c('KRT6C','GJB6','SBSN','KRTDAP','KRT6B'))     

features <- unlist(unname(gene_markers))

palette <- list("NON-SCC(p16+)" = '#F3766E',
                "CIN" = '#7CAF41',
                "SCC" = '#19BDC2',
                "NON-SCC" = '#A780BA')

 scale ="row"
 color = grDevices::colorRampPalette(c("navy", "white", "red"))(50)
 cluster_cols = T
 cluster_rows = F
 fontsize_row = 15
 fontsize_col = 15
 cutree_cols = 9
 silent = TRUE


col_annot <- data.frame(sample = edger_obj$samples$type)
row.names(col_annot) <- colnames(as.data.frame(edgeR::cpm(edger_obj,log=TRUE)))

annotation_colors <- list(sample = unlist(palette))


mtx <- as.matrix(as.data.frame(edgeR::cpm(edger_obj,log=TRUE))[unique(features),])


p <- pheatmap::pheatmap(mtx,scale=scale,color=color,cluster_cols = cluster_cols, annotation_col=col_annot, cluster_rows = cluster_rows,
                      fontsize_row = fontsize_row, fontsize_col = fontsize_col, cutree_cols = cutree_cols, silent = silent, annotation_colors = annotation_colors)


In [ ]:
rownames(mtx)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
p

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)

pdf(file = "~/CIN_subcluster_DE.pdf", height = 10, width = 10)
p
dev.off()

In [ ]:
features <- c(unlist(unname(gene_markers)), c("IL34", "CSF1R"))



palette <- list("NON-SCC(p16+)" = '#F3766E',
                "CIN" = '#7CAF41',
                "SCC" = '#19BDC2',
                "NON-SCC" = '#A780BA')

 scale ="row"
 color = grDevices::colorRampPalette(c("navy", "white", "red"))(50)
 cluster_cols = T
 cluster_rows = F
 fontsize_row = 15
 fontsize_col = 15
 cutree_cols = 9
 silent = TRUE


col_annot <- data.frame(sample = edger_obj$samples$type)
row.names(col_annot) <- colnames(as.data.frame(edgeR::cpm(edger_obj,log=TRUE)))

annotation_colors <- list(sample = unlist(palette))


mtx <- as.matrix(as.data.frame(edgeR::cpm(edger_obj,log=TRUE))[unique(features),])
mtx <- rbind(mtx,mtx["CSF1R",]*mtx["IL34",])
rownames(mtx) <- c(rownames(mtx)[1:length(rownames(mtx))-1], "CSF1R-IL34")


p <- pheatmap::pheatmap(mtx,scale=scale,color=color,cluster_cols = cluster_cols, annotation_col=col_annot, cluster_rows = cluster_rows,
                      fontsize_row = fontsize_row, fontsize_col = fontsize_col, cutree_cols = cutree_cols, silent = silent, annotation_colors = annotation_colors)


In [ ]:
mtx

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)

pdf(file = "~/CIN_subcluster_CSF1R.pdf", height = 10, width = 10)
p
dev.off()

## Find DE Genes between Clusters

In [ ]:
DefaultAssay(int.data) <- "SCT"
de <- PrepSCTFindMarkers(int.data)
Idents(de) <- "cluster_de"
de <- FindAllMarkers(de, only.pos = T)

In [ ]:
de[de$cluster == "CIN",][1:10,]

In [ ]:
de[de$cluster == "NON-SCC",][1:10,]

In [ ]:
de[de$cluster == "SCC",][1:10,]

In [ ]:
int.data <- AddModuleScore(object =  int.data, features = list("cin" = c('SERPINB13','SERPINB5','CLDN1','TP63','CDKN2A','TNS4','DSC3')), name = "CIN_genes", assay = "SCT")
int.data <- AddModuleScore(object =  int.data, features = list("scc" = c('CASP14','PSORS1C2','DNAH17','SERPINB12','SLC44A5','TCHH','PNLDC1','DIAPH3','CPA4','CALML5' )), name = "SCC_genes", assay = "SCT")
int.data <- AddModuleScore(object =  int.data, features = list("normal" = c('KRT6C','GJB6','SBSN','KRTDAP','KRT6B')), name = "normal_genes", assay = "SCT")             

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 8)
FeaturePlot(int.data, features = c("CIN_genes1", "SCC_genes1", "normal_genes1"), ncol = 3)

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 8)
VlnPlot(int.data, features = "CIN_genes1", group.by = "integrated_snn_res.0.3", split.by = "cluster_de", assay = "SCT")
VlnPlot(int.data, features = "SCC_genes1", group.by = "integrated_snn_res.0.3", split.by = "cluster_de", assay = "SCT")
VlnPlot(int.data, features = "normal_genes1", group.by = "integrated_snn_res.0.3", split.by = "cluster_de", assay = "SCT")

In [ ]:
colnames(head(int.data))

In [ ]:
updated.metadata <- readRDS("~/final_int_data_list.rds")

In [ ]:
updated.metadata <- merge(updated.metadata[[1]], updated.metadata[2:length(updated.metadata)])

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
p <- DotPlot(int.data, group.by = "cluster_de",features = c('t_regulatory1','t_proinflammatory1','t_anti1','m1_signature1','m2_signature1','apc_langerhan1','lr_pair1'), col.min = 0)+ theme(panel.spacing = unit(x = 1, units = "lines"),strip.background = element_blank(),strip.text.x = element_text(angle = 90, hjust = 0),axis.text.x = element_text(angle = 90, hjust = 1))
p


In [ ]:
int.data@meta.data[c('t_regulatory1','t_proinflammatory1','t_anti1','m1_signature1','m2_signature1','apc_langerhan1','lr_pair1')] <- updated.metadata@meta.data[c('t_regulatory1','t_proinflammatory1','t_anti1','m1_signature1','m2_signature1','apc_langerhan1','lr_pair1')]

In [ ]:
data_list[[3]] <- AddModuleScore(object =  data_list[[3]], features = list("cin" = c('SERPINB13','SERPINB5','CLDN1','TP63','CDKN2A','TNS4','DSC3')), name = "CIN_genes", assay = "SCT", nbin = 5)    
data_list[[3]] <- AddModuleScore(object =  data_list[[3]], features = list("normal" = c('KRT6C','GJB6','SBSN','KRTDAP','KRT6B')), name = "normal_genes", assay = "SCT", nbin = 5)             

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 5)
p <- DotPlot(subset(int.data, subset = integrated_snn_res.0.3 == 8), group.by = "cluster_de",features = c('t_regulatory1','t_proinflammatory1','t_anti1','m1_signature1','m2_signature1','apc_langerhan1','lr_pair1')#, col.min = 0
            )+ theme(panel.spacing = unit(x = 1, units = "lines"),strip.background = element_blank(),strip.text.x = element_text(angle = 90, hjust = 0),axis.text.x = element_text(angle = 90, hjust = 1))
p
ggsave(p, filename = "~/CIN_MS_dotplots_clust8.pdf", height = 5, width = 8)

In [ ]:
combined_sores <- lapply(c('t_regulatory1','t_proinflammatory1','t_anti1','m1_signature1','m2_signature1','apc_langerhan1'), function(x){
    ifelse(int.data@meta.data[[x]] & int.data$lr_pair1 > 0, "pos", "neg")})

combined <- do.call(cbind,combined_sores)
int.data@meta.data[paste0(c('t_regulatory1','t_proinflammatory1','t_anti1','m1_signature1','m2_signature1','apc_langerhan1'), "_comb")] <- combined


In [ ]:
options(repr.plot.width = 25, repr.plot.height = 10)
lapply(paste0(c('t_regulatory1','t_proinflammatory1','t_anti1','m1_signature1','m2_signature1','apc_langerhan1'), "_comb"), function(x){
    SpatialDimPlot(int.data, group.by = x, pt.size.factor = 2)})


In [ ]:
options(repr.plot.width = 8, repr.plot.height = 5)
lapply(paste0(c('t_regulatory1','t_proinflammatory1','t_anti1','m1_signature1','m2_signature1','apc_langerhan1'), "_comb"), function(x){
    df_long <- data.frame(table(int.data$cluster_de, int.data@meta.data[[x]]))
    df_prop <- df_long %>%
      group_by(Var1) %>%
      mutate(prop = Freq / sum(Freq))

    ggplot(df_prop, aes(x = Var1, y = prop, fill = Var2)) +
      geom_bar(stat = "identity") +
      scale_y_continuous(labels = scales::percent_format()) +
      labs(x = "Group", y = "Proportion", fill = "Status") +
      theme_minimal() +
      theme(axis.text.x = element_text(angle = 45, hjust = 1))+ggtitle(x)

    })

## Look at IL34 in Tumour Subcluster

In [ ]:
cluster8 <- subset(int.data , subset = integrated_snn_res.0.3 == 8)

In [ ]:
cluster8$ILpos <- ifelse( cluster8$IL34_CSF1R > 0, "IL34-CSF1R +", "IL34-CSF1R -")

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
p <- VlnPlot(cluster8, features = "lr_pair1", group.by = "cluster_annotations")

p


In [ ]:
options(repr.plot.width = 20, repr.plot.height = 15)

p <- SpatialDimPlot(cluster8, group.by = "ILpos", crop = F, pt.size.factor = 2, ncol = 3, cols = list("IL34-CSF1R +" = "red", "IL34-CSF1R -" = "blue"))
p

ggsave(p, filename = "~/CIN_subcluster_il34pos_spatial.pdf", height = 15, width = 20)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
DimPlot(int.data, group.by = "cluster_de")

## Niche Analysis

In [ ]:
source("https://raw.githubusercontent.com/agc888/Helper-Scripts-for-Bioinformatics/refs/heads/main/R%20Scripts/MultiSampleNicheAnalysis_Seurat")

In [ ]:
combined.niche <- BuildNicheAssay.using_all_fovs(int.data, group.by = "integrated_snn_res.0.3", niches.k = 8, platform = "Visium",neighbors.k = 6)

In [ ]:
niche_palette <- RColorBrewer::brewer.pal(name = "Paired", n = length(unique(combined.niche$niches)))
names(niche_palette) <- unique(combined.niche$niches)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
DimPlot(combined.niche, group.by = "cluster_de")|DimPlot(combined.niche, group.by = "niches", cols = niche_palette)

In [ ]:
#sub <- subset(combined.niche, subset = cluster_de %in% c("CIN", "NON-SCC", "SCC", "NON-SCC (p16+)"))
sub <- subset(combined.niche, subset = niches %in% c("4", "7", "8"))

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 8)
(SpatialDimPlot(combined.niche, group.by = "niches", ncol = 7, cols = niche_palette)& NoLegend())/
(SpatialDimPlot(sub, group.by = "niches", ncol = 7, crop = F, pt.size.factor = 2 ,cols = niche_palette)& NoLegend())

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
df <- data.frame(table(combined.niche$cluster_de, combined.niche$niches))
df$Var1 <- factor(df$Var1, levels = unique(df$Var1))  # Ensure correct order
df$Var2 <- factor(df$Var2)  # Treat as categorical
ggplot(df, aes(x = Var2, y = Var1, fill = Freq)) +
  geom_tile() +
  scale_fill_viridis_c(option = "magma", direction = -1) +  # Use Viridis colors
  theme_minimal() +
  labs(x = "Category (Var2)", y = "Cell Type (Var1)", fill = "Frequency") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

In [ ]:
library(dplyr)
library(pheatmap)

# Convert to matrix format for pheatmap
heatmap_matrix <- df %>%
  select(Var1, Var2, Freq) %>%
  tidyr::spread(key = Var2, value = Freq)  # Wide format

# Convert Var1 to row names and remove it from columns
rownames(heatmap_matrix) <- heatmap_matrix$Var1
heatmap_matrix <- heatmap_matrix[, -1]  # Drop Var1 column

# Convert to matrix
heatmap_matrix <- as.matrix(heatmap_matrix)
p <- pheatmap(heatmap_matrix,
         #color = viridis::viridis(100, option = "magma"),
         cluster_rows = TRUE,    # Cluster similar rows (optional)
         cluster_cols = TRUE,   # Keep Var2 in order
         scale = "column",         # Already scaled
         border_color = NA,      # Remove grid lines
         fontsize_row = 10,
         fontsize_col = 10)

p

## Trajectory

In [ ]:
colnames(head(int.data))

In [ ]:
library(monocle3)
seurat_to_cds_monocle3 <- function(seurat_obj, assay="Spatial") {
  expression_matrix <- Seurat::GetAssayData(seurat_obj, slot = "count", assay=assay)  # or use "data" for normalized counts
  cell_metadata <- seurat_obj@meta.data
  cell_metadata$cells <- rownames(seurat_obj@meta.data)
  gene_metadata <- data.frame(
    gene_short_name = rownames(expression_matrix),
    row.names = rownames(expression_matrix)
  )
  cds <- monocle3::new_cell_data_set(
    expression_data = expression_matrix,
    cell_metadata = cell_metadata,
    gene_metadata = gene_metadata
  )
  return(cds)
}


In [ ]:
sub <- subset(int.data, subset = integrated_snn_res.0.3 == 8)

In [ ]:
xen <- seurat_to_cds_monocle3(int.data, assay="SCT")
xen <- preprocess_cds(xen, method = "PCA", num_dim = 30, norm_method = "none")
xen <- align_cds(xen, alignment_group = "sample")
xen <- reduce_dimension(xen)
xen <- cluster_cells(xen, reduction_method = "UMAP")

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 8)
plot_cells(xen, color_cells_by = "partition")|plot_cells(xen, color_cells_by = "integrated_snn_res.0.3")|plot_cells(xen, color_cells_by = "cluster_de", group_label_size = 3)
DimPlot(int.data, group.by = "sample")|DimPlot(int.data, group.by = "integrated_snn_res.0.3")|DimPlot(int.data, group.by = "cluster_de")

In [ ]:
xen <- learn_graph(xen)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
plot_cells(xen,
           color_cells_by = "cluster_de",
           label_groups_by_cluster=F,
           label_leaves=TRUE,
           label_branch_points=T,
           label_principal_points = T, 
           graph_label_size = 5,
           group_label_size = 1, cell_size = 1)

In [ ]:
xen <- monocle3::order_cells(xen, reduction_method = "UMAP", root_pr_nodes="Y_146")

In [ ]:
p <-plot_cells(xen,
           color_cells_by = "pseudotime",
           label_groups_by_cluster=F,
           label_leaves=FALSE,
           label_branch_points=F,
           label_principal_points = F, 
           graph_label_size = 5,
           group_label_size = 5, cell_size = 1)+  scale_color_gradientn(colours =  rev(viridis::plasma(10)))


p1 <- plot_cells(xen,
           color_cells_by = "cluster_de",
           label_groups_by_cluster=F,
           label_leaves=FALSE,
           label_branch_points=F,
           label_principal_points = F, 
           graph_label_size = 5,
           group_label_size = 5, cell_size = 1)

options(repr.plot.width = 15, repr.plot.height = 8)
p|p1

In [ ]:
ggsave((p|p1), filename = "~/CIN_UMAP_monocle.pdf", height = 8, width = 15)

In [ ]:
#ciliated_cds_pr_test_res <- graph_test(xen, neighbor_graph="principal_graph", cores=4)
#pr_deg_ids <- row.names(subset(ciliated_cds_pr_test_res, q_value < 0.05))

In [ ]:
ciliated_cds_pr_test_res[order(-ciliated_cds_pr_test_res$morans_I),]

In [ ]:
# Extract pseudotime values
pseudotime_values <- xen@principal_graph_aux$UMAP$pseudotime

# Match rownames from xenium_annotated to the names in pseudotime_values
matching_indices <- match(rownames(int.data@meta.data), names(pseudotime_values))

# Create a vector of NAs of the same length as xenium_annotated metadata
pseudotime_filled <- rep(NA, nrow(int.data@meta.data))

# Assign matched values
pseudotime_filled[!is.na(matching_indices)] <- pseudotime_values[matching_indices[!is.na(matching_indices)]]

# Store in Seurat metadata
int.data$pseudotime <- pseudotime_filled

rows_with_inf <- apply(data.frame(int.data$pseudotime), 1, function(x) any(is.infinite(x)))

int.data$pseudotime[rows_with_inf] <- NA          


In [ ]:
options(repr.plot.width = 18, repr.plot.height = 8)

p <- FeaturePlot(int.data, features = "pseudotime")+  scale_color_viridis_c(option = "plasma")|DimPlot(int.data, group.by = "cluster_de")
p

ggsave(p , filename = "~/CIN_UMAP_pseudotime_cluster.pdf", height = 8, width = 18)

In [ ]:
sub <- subset(int.data, subset = integrated_snn_res.0.3 == 8)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)

p <- FeaturePlot(sub, features = "pseudotime")+  scale_color_viridis_c(option = "plasma")
p

ggsave(p , filename = "~/CIN_UMAP_pseudotime_cluster8.pdf", height = 6, width = 6)

In [ ]:
int.data$pseudotime_adj <- ifelse(is.na(int.data$pseudotime), 0, int.data$pseudotime+1)

In [ ]:
int.data$pseudotime_bins<- ifelse(int.data$pseudotime_adj == 0, "NA", 
       ifelse(int.data$pseudotime_adj >0 & int.data$pseudotime_adj <= 4, "0-3", 
            ifelse(int.data$pseudotime_adj >4 & int.data$pseudotime_adj <= 7, "4-6", 
              ifelse(int.data$pseudotime_adj >7 & int.data$pseudotime_adj <= 11, "7-10",
              ifelse(int.data$pseudotime_adj >11 &int.data$pseudotime_adj <= 21, "11-20",
                                   ifelse(int.data$pseudotime_adj >21 &int.data$pseudotime_adj <= 31, "21-30",
                                                                             "30+"))))))

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 10)
SpatialDimPlot(int.data, group.by = "pseudotime_bins")

In [ ]:
ciliated_cds_pr_test_res[order(-ciliated_cds_pr_test_res$morans_I),]["VIM",]
ciliated_cds_pr_test_res[order(-ciliated_cds_pr_test_res$morans_I),]["IL34",]
ciliated_cds_pr_test_res[order(-ciliated_cds_pr_test_res$morans_I),]["CSF1R",]

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 10)
p <- SpatialFeaturePlot(int.data, features = "pseudotime", pt.size.factor = 2) &  scale_fill_gradientn(colours =  rev(viridis::plasma(10)))
p
ggsave(p, filename = "~/CIN_spatial_pseudotime.pdf", height = 10, width = 30)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)

p <- FeaturePlot(int.data, features = "pseudotime")+  scale_color_viridis_c(option = "plasma")
ggsave(p, filename = "~/CIN_UMAP_pseudotime.pdf", height = 8, width = 8)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)

p <- plot_cells(xen,
           color_cells_by = "pseudotime",
           label_groups_by_cluster=F,
           label_leaves=FALSE,
           label_branch_points=F,
           label_principal_points = F, 
           graph_label_size = 5,
           group_label_size = 5, cell_size = 1)
p
ggsave(p, filename = "~/CIN_UMAP_pseudotime_monocle.pdf", height = 8, width = 8)

In [ ]:
RidgePlot(int.data, features = "pseudotime", group.by = "cluster_de")

In [ ]:
RidgePlot(int.data, features = "IL34_CSF1R", group.by = "cluster_de")

In [ ]:
AFD_genes <- c("IL34", "CSF1R", "CDKN2A", "IGFBP5")
               #"TP63","Epithelial_MS", "Cancer_MS"  ,"TCells_MS","TReg_MS",  "TPro_MS","TAnti_MS","Macro",  "Macro1_MS",
               #"Macro2_MS", "Langherhans_MS", "APC_Macro_MS", "IL34_CSF1R")


assay(xen[rowData(xen)$gene_short_name %in% AFD_genes,], "counts")


#AFD_lineage_cds <- order_cells(AFD_lineage_cds)

In [ ]:
t(int.data@meta.data[metadata_genes])

In [ ]:
traj <- xen
# Get the current counts matrix
#mat <- assay(traj, "counts")

metadata_genes <- c('t_regulatory1','t_proinflammatory1','t_anti1','m1_signature1','m2_signature1','apc_langerhan1','lr_pair1')

AFD_genes <- c("IGFBP5","CSF1R", "CDKN2A", "IL34")
#genes <- AFD_genes
genes <- c(metadata_genes, AFD_genes)


# Create a new row for IL34_CSF1R
#new_gene_x <- lapply(metadata_genes, function(x) {matrix(colData(traj)[[x]], nrow = 1)})
new_gene_y <- t(int.data@meta.data[metadata_genes])
#new_gene_x <- traj@principal_graph_aux$UMAP$pseudotime
#new_gene_x <- assay(traj, "counts")["IL34",]*assay(traj, "counts")["CDKN2A",]


new_gene_expr <- rbind(new_gene_y, assay(traj[rowData(traj)$gene_short_name %in% AFD_genes,], "counts"))

#new_gene_expr <- assay(traj[rowData(traj)$gene_short_name %in% AFD_genes,], "counts")

rownames(new_gene_expr) <- genes
colnames(new_gene_expr) <- rownames(colData(traj))

cds <- new_cell_data_set(Matrix(new_gene_expr, sparse = TRUE),
                         cell_metadata = colData(traj),
                         gene_metadata = NULL)

cds@principal_graph <- traj@principal_graph
cds@principal_graph_aux <- traj@principal_graph_aux
cds@reduce_dim_aux <- traj@reduce_dim_aux
cds@clusters <- traj@clusters
rowData(cds)$gene_short_name <- genes

In [ ]:
plots <- lapply(genes, function(x){to_plot <- cds[rowData(cds)$gene_short_name == x,]
                                  plot_genes_in_pseudotime(to_plot,
                                 color_cells_by="cluster_de",
                                     min_expr=0.5)
                                  })


In [ ]:
options(repr.plot.width = 20, repr.plot.height = 20)
wrap_plots(plots, ncol = 2)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)

p <- wrap_plots(plots[c(3,2,1,4)], ncol = 2)
p


## Trajectory Sample3

In [ ]:
xen <- seurat_to_cds_monocle3(data_list[[3]], assay="SCT")
xen <- preprocess_cds(xen, method = "PCA", num_dim = 30, norm_method = "none")
xen <- align_cds(xen)
xen <- reduce_dimension(xen)
xen <- cluster_cells(xen, reduction_method = "UMAP")

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 8)
plot_cells(xen, color_cells_by = "partition")|plot_cells(xen, color_cells_by = "integrated_snn_res.0.3")|plot_cells(xen, color_cells_by = "cluster_de", group_label_size = 3)
DimPlot(int.data, group.by = "sample")|DimPlot(int.data, group.by = "integrated_snn_res.0.3")|DimPlot(int.data, group.by = "cluster_de")

In [ ]:
xen <- learn_graph(xen)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
plot_cells(xen,
           color_cells_by = "cluster_de",
           label_groups_by_cluster=F,
           label_leaves=TRUE,
           label_branch_points=T,
           label_principal_points = T, 
           graph_label_size = 5,
           group_label_size = 1, cell_size = 1)

In [ ]:
xen <- monocle3::order_cells(xen, reduction_method = "UMAP", root_pr_nodes="Y_95")

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
plot_cells(xen,
           color_cells_by = "pseudotime",
           label_groups_by_cluster=F,
           label_leaves=FALSE,
           label_branch_points=F,
           label_principal_points = F, 
           graph_label_size = 5,
           group_label_size = 5, cell_size = 1)


In [ ]:
# Extract pseudotime values
pseudotime_values <- xen@principal_graph_aux$UMAP$pseudotime

# Match rownames from xenium_annotated to the names in pseudotime_values
matching_indices <- match(rownames(data_list[[3]]@meta.data), names(pseudotime_values))

# Create a vector of NAs of the same length as xenium_annotated metadata
pseudotime_filled <- rep(NA, nrow(data_list[[3]]@meta.data))

# Assign matched values
pseudotime_filled[!is.na(matching_indices)] <- pseudotime_values[matching_indices[!is.na(matching_indices)]]

# Store in Seurat metadata
data_list[[3]]$pseudotime <- pseudotime_filled

rows_with_inf <- apply(data.frame(data_list[[3]]$pseudotime), 1, function(x) any(is.infinite(x)))

data_list[[3]]$pseudotime[rows_with_inf] <- NA          

#xenium_annotated$pseudotime_adj <- ifelse(is.na(xenium_annotated$pseudotime), 0, xenium_annotated$pseudotime+1)

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 8)

DimPlot(data_list[[3]], group.by = "integrated_snn_res.0.3")|FeaturePlot(data_list[[3]], features = "pseudotime")+  scale_color_viridis_c(option = "plasma")|DimPlot(data_list[[3]], group.by = "cluster_de")

In [ ]:
head(data_list[[3]])

In [ ]:
sub <- subset(data_list[[3]], subset = cluster8_ind == "Other", invert = T)
VlnPlot(sub, features = "pseudotime",group.by = "cluster8_ind")|SpatialFeaturePlot(data_list[[3]], features = "pseudotime", pt.size.factor = 2.5)|
SpatialDimPlot(data_list[[3]], group.by = "cluster8_ind", pt.size.factor = 2.5)

In [ ]:
ciliated_cds_pr_test_res <- graph_test(xen, neighbor_graph="principal_graph", cores=4)
pr_deg_ids <- row.names(subset(ciliated_cds_pr_test_res, q_value < 0.05))

In [ ]:
ciliated_cds_pr_test_res[order(-ciliated_cds_pr_test_res$morans_I),]

In [ ]:
data_list[[3]] <- AddModuleScore(object =  data_list[[3]], features = list("cin" = c('SERPINB13','SERPINB5','CLDN1','TP63','CDKN2A','TNS4','DSC3')), name = "CIN_genes", assay = "SCT", nbin = 5)    
data_list[[3]] <- AddModuleScore(object =  data_list[[3]], features = list("normal" = c('KRT6C','GJB6','SBSN','KRTDAP','KRT6B')), name = "normal_genes", assay = "SCT", nbin = 5)             

In [ ]:
options(repr.plot.width = 35, repr.plot.height = 8)
p  <- (SpatialDimPlot(data_list[[3]], group.by = "cluster8_ind", pt.size.factor = 2.5)|
SpatialFeaturePlot(data_list[[3]], features = "pseudotime", pt.size.factor = 2.5)|
SpatialFeaturePlot(data_list[[3]], features = "IGFBP5", pt.size.factor = 2.5)|
SpatialFeaturePlot(data_list[[3]], features = "normal_genes1", pt.size.factor = 2.5)|
SpatialFeaturePlot(data_list[[3]], features = "CIN_genes1", pt.size.factor = 2.5))

p

## Public data

In [ ]:
public <- readRDS( "~/data_list.RDS")

In [ ]:
for (sample in names(public)){
    public[[sample]] <- SCTransform(public[[sample]], verbose = F)
}

In [ ]:
public_merged <- merge(public[[1]], public[2:length(public)])
public_merged <- JoinLayers(public_merged, assay = "RNA")

In [ ]:
public_merged$sample <- unlist(lapply(rownames(public_merged@meta.data), function(x) { strsplit(x, split = "_")[[1]][2]}))

In [ ]:
public_merged <- AddModuleScore(object =  public_merged, features = list("cin" = c('SERPINB13','SERPINB5','CLDN1','TP63','CDKN2A','TNS4','DSC3')), name = "CIN_genes", assay = "SCT")
public_merged <- AddModuleScore(object =  public_merged, features = list("scc" = c('CASP14','PSORS1C2','DNAH17','SERPINB12','SLC44A5','TCHH','PNLDC1','DIAPH3','CPA4','CALML5' )), name = "SCC_genes", assay = "SCT")
public_merged <- AddModuleScore(object =  public_merged, features = list("normal" = c('KRT6C','GJB6','SBSN','KRTDAP','KRT6B')), name = "normal_genes", assay = "SCT")             
public_merged <- AddModuleScore(object =  public_merged, features = list("cancer" = c("SERPINB3", "TP63", "KRT5", "CDKN2A")), name = "Cancer_MS", assay = "SCT")  
public_merged <- AddModuleScore(object =  public_merged, features = list("IL34_CSF1R" = c("IL34", "CSF1R")), name = "IL34_CSF1R", assay = "SCT")  

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)

public_data <- VlnPlot(public_merged, features = c("nCount_RNA", "nFeature_RNA"), group.by = "sample", ncol = 1, log = T)
inhouse_data <- VlnPlot(int.data, features = c("nCount_Spatial", "nFeature_Spatial"), group.by = "sample", ncol = 1, log = T)

In [ ]:
public_df <- public_data[[1]]$data
public_df$data <- public_df$nCount_RNA
public_df$sample <- "public"
public_df$nCount_RNA <- NULL
public_df$ident <- paste0(public_df$ident, "_public")
rownames(public_df) <- paste0(rownames(public_df), "_p")

inhouse_df <- inhouse_data[[1]]$data
inhouse_df$data <- inhouse_df$nCount_Spatial
inhouse_df$sample <- "in-house"
inhouse_df$nCount_Spatial <- NULL
inhouse_df$ident <- paste0(inhouse_df$ident, "_public")
rownames(inhouse_df) <- paste0(rownames(inhouse_df), "_h")

df <- rbind(public_df, inhouse_df)


In [ ]:
p1 <- ggplot(df, aes(x = factor(ident), y = data, fill = sample)) +
  geom_boxplot() +
  geom_hline(yintercept = median(int.data$nCount_Spatial), color = "red", linetype = "dashed") +
     scale_y_log10() +
  labs(x = "Ident Group", y = "nCount_RNA") +
  theme_classic()

In [ ]:
public_df <- public_data[[2]]$data
public_df$data <- public_df$nFeature_RNA
public_df$sample <- "public"
public_df$nFeature_RNA <- NULL
public_df$ident <- paste0(public_df$ident, "_public")
rownames(public_df) <- paste0(rownames(public_df), "_p")

inhouse_df <- inhouse_data[[2]]$data
inhouse_df$data <- inhouse_df$nFeature_Spatial
inhouse_df$sample <- "in-house"
inhouse_df$nFeature_Spatial <- NULL
inhouse_df$ident <- paste0(inhouse_df$ident, "_public")
rownames(inhouse_df) <- paste0(rownames(inhouse_df), "_h")

df <- rbind(public_df, inhouse_df)


p2 <- ggplot(df, aes(x = factor(ident), y = data, fill = sample)) +
  geom_boxplot() +
  geom_hline(yintercept = median(int.data$nFeature_Spatial), color = "red", linetype = "dashed") +
 #scale_y_log10() +
  labs(x = "Ident Group", y = "nCount_RNA") +
  theme_classic()

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 4)
p1
p2

In [ ]:
library(viridis)
samps <- unlist(lapply(names(public), function(x) {strsplit(x, "_")[[1]][2]}))
sizes <- c(3,2,2.3,2)

options(repr.plot.width = 15, repr.plot.height = 5)
patchwork::wrap_plots(lapply(names(public_merged@images), function(fov){ImageFeaturePlot(public_merged, fov = fov, features = "CDH16", size = sizes[which(names(public_merged@images) == fov)], dark.background = F)& coord_flip() & scale_x_reverse()& scale_fill_gradientn(colours = viridisLite::viridis(10), limits = c(0, 1),
                                                                                                                                                      na.value = viridis(10)[1]) & ggtitle(samps[which(names(public_merged@images) == fov)])}), ncol = 4)

patchwork::wrap_plots(lapply(names(public_merged@images), function(fov){ImageFeaturePlot(public_merged, fov = fov, features = "CDH17", size = sizes[which(names(public_merged@images) == fov)], dark.background = F)& coord_flip() & scale_x_reverse()& scale_fill_gradientn(colours = viridisLite::viridis(10), limits = c(0, 1),
                                                                                                                                                      na.value = viridis(10)[1]) & ggtitle(samps[which(names(public_merged@images) == fov)])}), ncol = 4)


patchwork::wrap_plots(lapply(names(public_merged@images), function(fov){ImageFeaturePlot(public_merged, fov = fov, features = "VSIG1", size = sizes[which(names(public_merged@images) == fov)], dark.background = F)& coord_flip() & scale_x_reverse()& scale_fill_gradientn(colours = viridisLite::viridis(10), limits = c(0, 1),
                                                                                                                                                      na.value = viridis(10)[1]) & ggtitle(samps[which(names(public_merged@images) == fov)])}), ncol = 4)


patchwork::wrap_plots(lapply(names(public_merged@images), function(fov){ImageFeaturePlot(public_merged, fov = fov, features = "CTSE", size = sizes[which(names(public_merged@images) == fov)], dark.background = F)& coord_flip() & scale_x_reverse()& scale_fill_gradientn(colours = viridisLite::viridis(10), limits = c(0, 1),
                                                                                                                                                      na.value = viridis(10)[1]) & ggtitle(samps[which(names(public_merged@images) == fov)])}), ncol = 4)


In [ ]:

options(repr.plot.width = 15, repr.plot.height = 7)
p <- VlnPlot(public_merged, features = c("CIN_genes1", "Cancer_MS1", "SCC_genes1", "IL34_CSF1R1"), group.by = "sample", ncol = 2)

ggsave(p , filename = "~/CIN_public_vlns.pdf", height = 7, width = 15)

In [ ]:
library(viridisLite)

samps <- unlist(lapply(names(public), function(x) {strsplit(x, "_")[[1]][2]}))
sizes <- c(3,2,2.3,2)

options(repr.plot.width = 20, repr.plot.height = 12)
p <- patchwork::wrap_plots(lapply(names(public_merged@images), function(fov){ImageFeaturePlot(public_merged, fov = fov, features = "Cancer_MS1", size = sizes[which(names(public_merged@images) == fov)], dark.background = F)& coord_flip() & scale_x_reverse()& scale_fill_gradientn(colours = viridisLite::viridis(10), limits = c(0, 1),
                                                                                                                                                      na.value = viridis(10)[1]) & ggtitle(samps[which(names(public_merged@images) == fov)])}), ncol = 4)
p2 <- patchwork::wrap_plots(lapply(names(public_merged@images), function(fov){ImageFeaturePlot(public_merged, fov = fov, features = "SCC_genes1", size = sizes[which(names(public_merged@images) == fov)], dark.background = F)& coord_flip() & scale_x_reverse()& scale_fill_gradientn(colours = viridisLite::viridis(10), limits = c(0, 1),
                                                                                                                                                      na.value = viridis(10)[1]) &ggtitle(samps[which(names(public_merged@images) == fov)])}), ncol = 4)

p3 <- patchwork::wrap_plots(lapply(names(public_merged@images), function(fov){ImageFeaturePlot(public_merged, fov = fov, features = "IL34_CSF1R1", size = sizes[which(names(public_merged@images) == fov)], dark.background = F)& coord_flip() & scale_x_reverse()& scale_fill_gradientn(colours = viridisLite::viridis(10), limits = c(0, 1),
                                                                                                                                                      na.value = viridis(10)[1]) &ggtitle(samps[which(names(public_merged@images) == fov)])}), ncol = 4)

p/p2/p3

ggsave((p/p2/p3) , filename = "~/CIN_public_Spatial_MSs.pdf", height = 12, width = 20)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 6)
p <- FeatureScatter(public_merged, feature1 = "CIN_genes1", "IL34_CSF1R1", split.by = "sample")
plot_data <- p[[1]]
plot_data$colors <- ifelse(plot_data$CIN_genes1 < 0, "red", "blue") 
# Make sure `sample` is a factor if you want ordered facets
plot_data$sample <- as.factor(plot_data$sample)

 p <- ggplot(plot_data, aes(x = CIN_genes1, y = IL34_CSF1R1, colour = colors)) +
  geom_point(size = 1) +
  geom_smooth(method = "lm", se = FALSE, colour = "black", linewidth = 0.5) +
     facet_wrap(~ sample, ncol = 4) +
    scale_colour_identity() +  # Uses the exact colours in `colors` column
  theme_classic() +
  labs(x = "CIN_genes1", y = "IL34_CSF1R1", title = "Per-sample Scatter Plot with Trend Line")

p 

ggsave(p , filename = "~/CIN_public_MSs.pdf", height = 6, width = 20)